In [19]:
import pandas as pd
import json
import numpy as np
from os import listdir
from io import StringIO
from Bio import Phylo
import re

In [20]:
path="/home/maeperez/scratch/Clams/hyphy2.5/gard"
path_out="/home/maeperez/projects/def-bacc/maeperez/Clams/VesicSymb_Evolution/dowstream_analyses_and_data/"

genes=[]
for file in listdir(path):
    if file[-4:]=="json":
        genes+=[str(file.split('.')[0])]
genes=sorted(genes)
print(len(genes))

638


In [21]:
with_recombination=[]
for gene in genes:
    remote_file = open(path+'/'+gene+'.GARD.json')
    gard_json = json.load(remote_file)
    
    if 'improvements' not in gard_json.keys():
        ### write gard_splits output for KH test (has to be done with the older version of hyphy? )
        entry='\n\n'
        entry+='-'.join(map(str,([i-1 for i in gard_json['breakpointData']['bps'][0]])))+'\n'+gard_json['breakpointData']['tree']+'\n'
        with open(path+'/'+gene+'.gard_splits','w') as fa:
            fa.write(entry) 
        continue
        
    else:
#         max_bps=[gard_json['improvements'][bp]['breakpoints'] for bp in list(gard_json['improvements'].keys())[-1]][0]
#         max_part=int(list(gard_json['improvements'].keys())[-1])+1

        ### write gard_splits output for KH test (has to be done with the older version of hyphy? )
        entry='\n\n'
        for n in gard_json['breakpointData'].keys():
            entry+='-'.join(map(str,([i-1 for i in gard_json['breakpointData'][str(n)]['bps'][0]])))+'\n'+gard_json['breakpointData'][str(n)]['tree']+'\n'
        with open(path+'/'+gene+'.gard_splits','w') as fa:
            fa.write(entry)   

        best_AICc=sum([gard_json['improvements'][bp]['deltaAICc'] for bp in gard_json['improvements'].keys()])+gard_json['baselineScore']
        AICc_ratio=float(gard_json['singleTreeAICc'])/best_AICc
        if AICc_ratio>=1:
            with_recombination+=[gene]
print( 'number of genes that failed AICc test ',len(with_recombination),"/",len(genes))        

number of genes that failed AICc test  143 / 638


In [25]:
with open(path_out+'core_genes_with_recomb_AICc.txt','w') as f:
    f.write('\n'.join(with_recombination))

In [22]:
%%bash
# Run KH test ...

# sbatch run_array_KHtest.sl

In [26]:
#### PARSE KH test results ####
## GROUP1 ##
path="/home/maeperez/scratch/Clams/hyphy2.5/gard/"
onlyfiles = [path+f for f in listdir(path) if f[-7:]=='.KHtest']
kh=pd.DataFrame([],columns=['Gene','p0.01','p0.05','p0.1'])
print( 'number of genes that ran the KH test',len(onlyfiles))

for i in range(len(onlyfiles)):
# for i in [0]:
    print(i)
    with open(onlyfiles[i]) as f:
        gene=re.split('/|\.|',onlyfiles[i])[-2]
        lines=f.read().splitlines()
        for n in reversed(range(len(lines))):
            if lines[n][:6]=='At p =':
#                 print lines[n-2]
#                 print lines[n-1]
#                 print lines[n]
                try:
                    p001= int(lines[n-2].split(' ')[-3])
                    p005= int(lines[n-1].split(' ')[-3])
                    p01= int(lines[n].split(' ')[-3])
                except:
                    p001= np.nan
                    p005= np.nan
                    p01= np.nan
                break
        newentry=dict(zip(['Gene','p0.01','p0.05','p0.1'],[gene,p001,p005,p01]))
        kh=kh.append(newentry,ignore_index=True)


print( 'number of genes that failed KH test ',len(kh.loc[kh['p0.05']>0]))
print( 'number of genes that failed AICc test ',len(with_recombination))
# number of genes that failed KH test  152
# number of genes that failed AICc test  143

number of genes that ran the KH test 638
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


/home/maeperez/virtualenv/py36/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
31

In [31]:
kh['AICctest']=0
kh.loc[kh['Gene'].isin(with_recombination),'AICctest']=1
kh

,Gene,p0.01,p0.05,p0.1,AICctest
0,Rmag_0947,1,1,1,0
1,Rmag_0113,0,0,0,1
2,Rmag_0834,1,1,1,0
3,Rmag_0188,NaN,NaN,NaN,0
4,Rmag_0592,0,0,0,1
...,...,...,...,...,...
633,Rmag_0837,1,1,1,0
634,Rmag_0944,1,1,1,0
635,Rmag_0285,0,0,0,0
636,Rmag_0604,0,0,0,0


In [42]:
# kh.to_csv(path_out+'gard_results.txt')
path_to_files='../mauve_alignments/fna/'
with open(path_to_files+'filelist_nopseudo_638','r') as f:
    lines=[file[8:-4] for file in f.read().splitlines()]

failed_kh=sorted(kh[kh.isna().any(axis=1)]['Gene'].values)
failed_kh_idx=[lines.index(gene)+1 for gene in failed_kh]

print('\n'.join([' '.join(map(str,item)) for item in list(zip(failed_kh_idx,failed_kh))]))
','.join(map(str,failed_kh_idx))


1 Rmag_0001
2 Rmag_0002
7 Rmag_0008
9 Rmag_0010
11 Rmag_0012
18 Rmag_0027
19 Rmag_0028
21 Rmag_0044
23 Rmag_0046
24 Rmag_0050
26 Rmag_0052
27 Rmag_0053
29 Rmag_0055
50 Rmag_0089
51 Rmag_0090
52 Rmag_0091
55 Rmag_0098
56 Rmag_0099
57 Rmag_0101
61 Rmag_0107
63 Rmag_0109
67 Rmag_0116
71 Rmag_0121
74 Rmag_0126
80 Rmag_0137
82 Rmag_0139
85 Rmag_0143
88 Rmag_0147
92 Rmag_0160
93 Rmag_0161
94 Rmag_0164
96 Rmag_0166
97 Rmag_0167
99 Rmag_0169
100 Rmag_0170
102 Rmag_0172
103 Rmag_0173
104 Rmag_0174
105 Rmag_0175
106 Rmag_0176
107 Rmag_0177
108 Rmag_0178
109 Rmag_0179
111 Rmag_0181
112 Rmag_0182
113 Rmag_0184
115 Rmag_0187
116 Rmag_0188
117 Rmag_0189
119 Rmag_0191
120 Rmag_0192
123 Rmag_0200
124 Rmag_0201
125 Rmag_0203
127 Rmag_0205
132 Rmag_0214
133 Rmag_0215
135 Rmag_0217
137 Rmag_0222
138 Rmag_0223
139 Rmag_0224
140 Rmag_0225
144 Rmag_0236
145 Rmag_0237
147 Rmag_0240
148 Rmag_0241
153 Rmag_0246
154 Rmag_0247
158 Rmag_0257
161 Rmag_0262
164 Rmag_0267
171 Rmag_0276
172 Rmag_0278
175 Rmag_0281
17

'1,2,7,9,11,18,19,21,23,24,26,27,29,50,51,52,55,56,57,61,63,67,71,74,80,82,85,88,92,93,94,96,97,99,100,102,103,104,105,106,107,108,109,111,112,113,115,116,117,119,120,123,124,125,127,132,133,135,137,138,139,140,144,145,147,148,153,154,158,161,164,171,172,175,176,178,179,180,184,185,187,193,195,196,209,219,221,222,226,232,233,234,238,239,253,259,263,264,269,272,275,280,283,285,286,289,292,293,294,301,303,306,308,310,312,314,327,328,332,333,334,338,339,343,382,386,401,404,415,418,434,439,443,456,495,502,507,515,525,541,542,545,549,552,554,569,589,590,599,604,616,618,623,625,629'

In [13]:
recomb_KH=kh.loc[kh['p0.05']>0]['Gene']
print('recomb with AICc only',len(set(with_recombination)-set(recomb_KH)))
print('recomb with KH only',len(set(recomb_KH)-set(with_recombination)))
print('recomb with KH and AICc',len(set(recomb_KH).intersection(set(with_recombination))))



recomb with AICc only 116
recomb with KH only 95
recomb with KH and AICc 27


In [39]:
print('%s/%d genes show evidence for recombination' % (len(with_recombination),len(genes)))
# 85/389 genes show evidence for recombination: hyphy2.5

0/508 genes show evidence for recombination


In [50]:
df=pd.read_csv('/home/maeperez/projects/def-bacc/maeperez/Clams/VesicSymb_Evolution/dowstream_analyses_and_data_ms1/KHtest_results.txt',sep=',',header=0,index_col=0)
df[df['Gene'].isin(with_recombination) & df['p0.05']>0]

print('genes in common between older version and hyphy2.5')
print(len(set(genes).intersection(set(df['Gene'].values))))
print('genes with signficant recomb in hyphy2.5 but not older version')
print(len(set(with_recombination)-set(df[df['p0.05']>0]['Gene'].values)))
print('genes with significant recomb older version but not hyphy 2.5')
print(len(set(df[df['Gene'].isin(genes) & df['p0.05']>0]['Gene'].values)-set(with_recombination)))
print('genes with significant recomb in both version')
print(len(set(with_recombination).intersection(set(df[df['p0.05']>0]['Gene'].values))))


genes in common between older version and hyphy2.5
351
genes with signficant recomb in hyphy2.5 but not older version
81
genes with significant recomb older version but not hyphy 2.5
30
genes with significant recomb in both version
4


In [62]:
print(len('.GTR.gard.json'))

14


In [56]:
print(path+'/'+gene+'_'+str(n)+'.tree')

/home/maeperez/scratch/Clams/hyphy2.5/gard/Rmag_0003_1.tree
